#### 1. Import required libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import wandb

#### 2. Initialize wandb

In [3]:
wandb.init(project="DA6401-Assignment-2")

#### 3. Apply data transformations

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

- Train transformations include random crops and flips for data augmentation
- Test transformations use a standardized center crop
- Both use normalization with ImageNet statistics

#### 4. Load datasets

In [ ]:
train_dataset = ImageFolder('nature_12K/inaturalist_12K/train', transform=train_transform)
test_dataset = ImageFolder('nature_12K/inaturalist_12K/val', transform=test_transform)

#### 5. Split data into train and validaton

In [6]:
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

#### 6. Create data loaders

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

#### 7. Load pre-trained ResNet50 model

In [8]:
model = torchvision.models.resnet50(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Ayaan/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:10<00:00, 10.2MB/s]


#### 8. Freeze early layers

In [9]:

for name, param in model.named_parameters():
    if "layer3" not in name and "layer4" not in name and "fc" not in name:
        param.requires_grad = False

Layers upto layer2 are frozen

#### 9. Replace the final fully connected layer

In [10]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # 10 classes in iNaturalist subset

#### 10. Move model to GPU if available

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

#### 11. Defining loss function and optimizer

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([
    {'params': model.layer3.parameters(), 'lr': 1e-4},
    {'params': model.layer4.parameters(), 'lr': 1e-4},
    {'params': model.fc.parameters(), 'lr': 1e-3}
])

- Uses cross-entropy loss for classification
- Sets different learning rates for different layer groups using Adam optimizer

#### 12. Training loop

In [13]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct / total
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()
    
    val_loss = val_loss / len(val_loader)
    val_acc = 100. * val_correct / val_total
    
    # Log metrics
    wandb.log({
        'epoch': epoch,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc
    })
    
    print(f'Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

Epoch 1/10: Train Loss: 1.0017, Train Acc: 66.90%, Val Loss: 0.8939, Val Acc: 71.55%
Epoch 2/10: Train Loss: 0.7887, Train Acc: 73.67%, Val Loss: 0.8536, Val Acc: 72.75%
Epoch 3/10: Train Loss: 0.6563, Train Acc: 78.42%, Val Loss: 0.7787, Val Acc: 75.00%
Epoch 4/10: Train Loss: 0.6095, Train Acc: 79.62%, Val Loss: 0.7256, Val Acc: 76.20%
Epoch 5/10: Train Loss: 0.5599, Train Acc: 81.42%, Val Loss: 0.8162, Val Acc: 76.05%
Epoch 6/10: Train Loss: 0.5328, Train Acc: 81.92%, Val Loss: 0.7355, Val Acc: 77.15%
Epoch 7/10: Train Loss: 0.5140, Train Acc: 82.42%, Val Loss: 0.8477, Val Acc: 75.15%
Epoch 8/10: Train Loss: 0.4760, Train Acc: 84.44%, Val Loss: 0.9347, Val Acc: 73.55%
Epoch 9/10: Train Loss: 0.4343, Train Acc: 85.74%, Val Loss: 0.8588, Val Acc: 75.15%
Epoch 10/10: Train Loss: 0.4186, Train Acc: 86.16%, Val Loss: 0.9246, Val Acc: 74.85%


- Trains the model for 10 epochs
- Tracks training and validation metrics
- Logs results to wandb
- Prints progress updates

#### 13. Testing the final model

In [14]:
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

test_acc = 100. * test_correct / test_total
print(f'Test Accuracy: {test_acc:.2f}%')
wandb.log({'test_acc': test_acc})

Test Accuracy: 81.20%


#### 14. Save the model

In [15]:
torch.save(model.state_dict(), 'resnet50_finetuned.pth')
wandb.finish()

epoch,▁▂▃▃▄▅▆▆▇█
test_acc,▁
train_acc,▁▃▅▆▆▆▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
val_acc,▁▂▅▇▇█▅▄▅▅
val_loss,▇▅▃▁▄▁▅█▅█
epoch,9
test_acc,81.2
train_acc,86.16077
train_loss,0.41864
val_acc,74.85
